# CDS services in Jupyter notebooks
<span style='color:blue'> Welcome to a Jupyter notebook demonstrating how to access some CDS services with Python. Here you will learn how to: </span>

- use the **Aladin Lite** widget, 
- find object info on **Simbad**,
- find and download a catalogue from **Vizier**, 
- crossmatch tables with **XMatch**

Find more tutorials @ https://github.com/cds-astro


## Aladin Lite & ipyaladin

**Aladin Lite** is a lightweight sky atlas running in the browser. It allows to: 

 - visualize sky maps
     - 900+ available, [`permanently updated`](https://aladin.u-strasbg.fr/hips/list)
 - overlay tabular data and footprints  
 
 
 **ipyaladin** is a widget developed at CDS, allowing for easy embedding of Aladin Lite into an IPython notebook.

In [6]:
import ipyaladin.aladin_widget as ipyal

aladin = ipyal.Aladin(target='PSZ1 G057.84+87.98', fov=1, survey='P/SDSS9/color')
aladin

Aladin(fov=1.0, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'ov…

In [5]:
# query object info on Simbad

from astroquery.simbad import Simbad

query_simbad = Simbad.query_objectids("Abell 1656")
query_simbad

ID
bytes21
IGR J12595+2755
[KRL2007b] 128
1H 1257+281
1M 1257+281
2A 1257+283
2E 2932
2U 1257+28
3A 1257+282
3U 1257+28


In [9]:
#find neighbors
query_simbad = Simbad.query_region('Abell 1656', radius='10m')
query_simbad
#display table in Aladin

aladin.add_table(query_simbad)

## VizieR in PyVO

[`VizieR`](https://vizier.u-strasbg.fr/viz-bin/VizieR) is an astronomical catalog service provided by CDS:

It can be accessed with Python scripts:

1. using `PyVO` and the Table Access Protocol (write your qyery in [`ADQL`](https://www.ivoa.net/documents/ADQL/20180112/PR-ADQL-2.1-20180112.html#tth_sEc4.2.11))

2. using the `Vizier` subpackage from the `astroquery` package (see a demo in [CDS_Notebook_1](https://github.com/hendhd/CDS-EAS2021/blob/main/webinars/CDS_Notebooks/CDS_Notebook_1.ipynb))
 

In [10]:
import pyvo 

tap_vizier = pyvo.dal.TAPService('http://tapvizier.u-strasbg.fr/TAPVizieR/tap')

query = """SELECT  *  FROM tap_schema.tables 
               WHERE table_name LIKE '%sdss%' """

list_sdss = tap_vizier.search(query).to_table()

list_sdss['table_name', 'description']

table_name,description
object,object
J/A+A/532/A74/sdss,"SDSS identifiers, redshift and confidence level for the EFIGI catalogue galaxies ( Baillard A., Bertin W., de Lapparent V., Fouque P., Arnouts S., Mellier Y., Pello R., Leborgne J.-F., Prugniel P., Makarov D., Marakova L., McCraken H.J., Bijaoui A., Tasca L.)"
J/A+A/499/395/sdss1353,"QSO SDSS 1353 photometric parameters for two images ( Paraficz D., Hjorth J., Eliasdottir A.)"
J/ApJ/780/92/sdss,"*SDSS photometry (table2) ( Szabo R., Ivezic Z., Kiss L.L., Kollath Z., Jones L., Sesar B., Becker A.C., Davenport J.R.A., Cutri R.M.)"
J/ApJS/177/103/sdss,"Spectroscopic properties of SDSS object ( Ho L.C., Darling J., Greene J.E.)"
sdssdr13_join,join sdss13 gaia ( Gaia collaboration)
J/A+A/569/A124/guv_sdss,"All GUViCS-SDSS matched point-like sources (J2) ( Voyer E.N., Boselli A., Boissier S., Heinis S., Cortese L., Ferrarese L., Cote P., Cuillandre J.-C., Gwyn S.D.J., Peng E.W., Zhang H., Liu C.)"
J/PAZh/44/124/sdss-bc,"SDSS binary candidates ( Khovritchev M.Yu., Apetyan A.A., Roshchina E.A., Izmailov I.S., Bikulova D.A., Ershova A.P., Balyaev I.A., Kulikova A.M., Petjur V.V., Shumilov A.A., Oskina K.I., L.A. Maksimova)"
II/306/sdss8,* output of the SDSS photometric catalog ( Adelman-McCarthy J.K. et al.)
J/ApJ/907/123/sdss_sz,"Revised S/Z catalog with annotations ( Iye M., Yagi M., Fukumoto H.)"


In [11]:
# Once you've chosen your catalogue,
# check what data are available 
# by querying only a small subsample
# and customize your Vizier query

query_test = """ SELECT TOP 10 * FROM "V/147/sdss12" """
test_sdss = tap_vizier.search(query_test).to_table()
test_sdss

RA_ICRS,DE_ICRS,mode,q_mode,class,SDSS12,m_SDSS12,SDSS-ID,objID,Sp-ID,SpObjID,parentID,flags,Status,e_RA_ICRS,e_DE_ICRS,ObsDate,Q,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,zsp,e_zsp,f_zsp,Vdisp,e_Vdisp,spInst,spType,spCl,subCl,spS/N,uFlags,us,uc,uDate,u'mag,e_u'mag,upmag,e_upmag,uPmag,e_uPmag,uPrad,e_uPrad,uoRA,uoDE,udVrad,udVell,uPA,gFlags,gs,gc,gDate,g'mag,e_g'mag,gpmag,e_gpmag,gPmag,e_gPmag,gPrad,e_gPrad,goRA,goDE,gdVrad,gdVell,gPA,rFlags,rs,rc,rDate,r'mag,e_r'mag,rpmag,e_rpmag,rPmag,e_rPmag,rPrad,e_rPrad,roRA,roDE,rdVrad,rdVell,rPA,iFlags,is,ic,iDate,i'mag,e_i'mag,ipmag,e_ipmag,iPmag,e_iPmag,iPrad,e_iPrad,ioRA,ioDE,idVrad,idVell,iPA,zFlags,zs,zc,zDate,z'mag,e_z'mag,zpmag,e_zpmag,zPmag,e_zPmag,zPrad,e_zPrad,zoRA,zoDE,zdVrad,zdVell,zPA,pmRA,e_pmRA,pmDE,e_pmDE,sigRA,sigDE,M,N,g(O),r(E),g(J),r(F),i(N),zph,e_zph,<zph>,chi2,u'Mag,g'Mag,r'Mag,i'Mag,z'Mag
deg,deg,,,,,,,,,,,,,arcsec,arcsec,yr,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,,km / s,km / s,,,,,,,,,yr,mag,mag,mag,mag,mag,mag,arcsec,arcsec,arcsec,arcsec,arcsec,arcsec,deg,,,,yr,mag,mag,mag,mag,mag,mag,arcsec,arcsec,arcsec,arcsec,arcsec,arcsec,deg,,,,yr,mag,mag,mag,mag,mag,mag,arcsec,arcsec,arcsec,arcsec,arcsec,arcsec,deg,,,,yr,mag,mag,mag,mag,mag,mag,arcsec,arcsec,arcsec,arcsec,arcsec,arcsec,deg,,,,yr,mag,mag,mag,mag,mag,mag,arcsec,arcsec,arcsec,arcsec,arcsec,arcsec,deg,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,,,,,,,,,,,,mag,mag,mag,mag,mag
float64,float64,int16,str1,int16,object,str1,object,int64,object,int64,int64,int64,int16,float32,float32,float64,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,int16,float64,float32,object,object,object,object,float32,int64,int16,int16,float64,float32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float32,float32,int64,int16,int16,float64,float32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float32,float32,int64,int16,int16,float64,float32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float32,float32,int64,int16,int16,float64,float32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float32,float32,int64,int16,int16,float64,float32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32,float32,float32,float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64
45.001596,0.003649,2,,3,J030000.38+000013.1,*,7264-301-5-0174-0618,1237676673934951018,,0,0,281543964623104,1025,0.187,0.183,2008.0006,3,25.902,0.807,24.61,0.947,22.075,0.182,21.593,0.185,21.468,0.474,--,--,--,--,--,,,,,--,9007199397380353,0,3,2008.0006,25.902,0.807,25.762,0.627,24.658,3.777,2.97,-1000.0,0.01,-0.08,0.0,0.132,33.2,9007199397380353,0,3,2008.0006,24.61,0.947,24.628,0.676,26.262,1.606,2.971,-1000.0,0.04,-0.06,0.856,0.326,117.4,4503874782134528,0,3,2008.0006,22.075,0.182,22.604,0.19,21.733,0.365,2.97,-1000.0,0.0,0.0,2.18,0.268,98.0,275280592896,0,3,2008.0006,21.593,0.185,22.089,0.188,21.604,0.5,2.875,0.776,0.31,-0.12,2.185,0.05,64.1,9007199388992769,0,3,2008.0006,21.468,0.474,21.705,0.355,21.041,0.96,11.307,-1000.0,0.02,-0.07,29.669,0.05,18.0,--,--,--,--,--,--,-1,-1,--,--,--,--,--,--,--,--,--,--,--,--,--,--
45.001673,0.003537,2,,3,J030000.40+000012.7,,4145-301-4-0411-0515,1237663277410615811,,0,0,281543964623104,1025,0.166,0.216,2003.7411,3,25.817,1.021,23.538,0.404,22.285,0.193,21.703,0.178,22.31,0.986,--,--,--,--,--,,,,,--,9007199388958721,1,6,2003.7411,25.817,1.021,24.709,1.341,24.811,2.825,0.765,0.354,0.06,-0.05,6.386,0.98,43.8,9007199388992769,0,3,2003.7411,23.538,0.404,23.565,0.306,22.982,0.523,7.36,-1000.0,0.06,-0.03,2.325,0.05,3.4,4503874773712896,0,3,2003.7411,22.285,0.193,22.697,0.192,23.25,1.24,2.037,0.376,0.0,0.0,1.099,0.05,99.5,275146342400,0,3,2003.7411,21.703,0.178,22.03,0.173,23.286,1.378,1.453,0.286,0.01,0.39,0.944,

In [12]:
#inspect data
test_sdss['class']

3
3
3
3
3
6
3
3
3
3


In [ ]:
# now customise your Vizier query

query = """ SELECT RA_ICRS, DE_ICRS, rmag, zsp
            FROM "V/147/sdss12"
            WHERE class=3 AND Q=3 AND rmag>11.5 AND rmag<17 AND zsp>0
            """
cat_sdss = tap_vizier.search(query).to_table()
cat_sdss

## XMatch

[`Crossmatching of big tables`](http://cdsxmatch.u-strasbg.fr/)

- Crossmatch any available VizieR table
- or upload your own tables

In [15]:
from astroquery.xmatch import XMatch
import astropy.units as u

xmatch_sdss_rines = XMatch.query(cat1='vizier:V/147/sdss12', 
                                 cat2='vizier:J/AJ/126/2152/galaxies',
                                 max_distance=1 * u.arcsec, 
                                 colRA1='RA_ICRS',
                                 colDec1='DE_ICRS',colRA2='RA_J2000',
                                 colDec2='DE_J2000')
xmatch_sdss_rines

angDist,RAdeg,DEdeg,errHalfMaj,errHalfMin,errPosAng,objID,mode,q_mode,class,SDSS12,m_SDSS12,flags,ObsDate,Q,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,zsp,e_zsp,f_zsp,zph,e_zph,avg_zph,pmRA,e_pmRA,pmDE,e_pmDE,SpObjID,spType,spCl,subClass,_RAJ2000,_DEJ2000,RAJ2000,DEJ2000,cz,e_cz,r_cz,Cluster
float64,float64,float64,float64,float64,int64,int64,int64,str1,int64,str19,str1,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,int64,str18,str6,str21,float64,float64,str11,str11,int64,int64,int64,str5
0.288917,9.121848,-0.728554,0.003,0.002,90,1237646010550255653,2,--,3,J003629.24-004342.7,*,35287719874576,1998.7333,3,17.751,0.017,15.872,0.003,15.061,0.002,14.668,0.002,14.366,0.004,--,--,--,--,--,--,-12.2,2.7,-3.2,2.7,0,--,--,--,9.1217917,-0.7286111,00 36 29.23,-00 43 43.0,10214,51,2,A0119
0.324371,9.121864,-0.728557,0.002,0.001,90,1237663783127613510,1,+,3,J003629.24-004342.8,*,35253360136208,2003.8855,3,17.762,0.015,15.906,0.003,15.08,0.003,14.708,0.003,14.388,0.004,0.03409,1e-05,0,0.04205,0.01104,0.04263,-9.6,2.4,-3.4,2.4,441389077447600128,GALAXY,GALAXY,--,9.1217917,-0.7286111,00 36 29.23,-00 43 43.0,10214,51,2,A0119
0.340962,9.121874,-0.728564,0.002,0.002,0,1237667171856416774,2,--,3,J003629.24-004342.8,*,35253360136208,2004.9505,3,17.772,0.016,15.916,0.003,15.084,0.003,14.665,0.003,14.391,0.004,--,--,--,--,--,--,-8.9,2.4,-3.9,2.4,0,--,--,--,9.1217917,-0.7286111,00 36 29.23,-00 43 43.0,10214,51,2,A0119
0.345813,9.121881,-0.728575,0.002,0.002,90,1237663276321144858,2,--,3,J003629.25-004342.8,--,35253360136208,2003.7408,3,17.756,0.016,15.912,0.003,15.08,0.003,14.682,0.003,14.395,0.004,--,--,--,--,--,--,-8.6,2.4,-4.9,2.4,0,--,--,--,9.1217917,-0.7286111,00 36 29.23,-00 43 43.0,10214,51,2,A0119
0.358494,9.121868,-0.728547,0.002,0.002,90,1237660222615322639,2,--,3,J003629.24-004342.7,*,35253360136208,2002.8289,3,17.73,0.016,15.906,0.003,15.083,0.003,14.696,0.003,14.391,0.004,--,--,--,--,--,--,-9.1,2.5,-3.1,2.5,0,--,--,--,9.1217917,-0.7286111,00 36 29.23,-00 43 43.0,10214,51,2,A0119
0.363257,9.121864,-0.72854,0.002,0.002,90,1237663525448581133,2,--,3,J003629.24-004342.7,*,35253360136208,2003.8091,3,17.742,0.017,15.898,0.003,15.081,0.003,14.696,0.003,14.383,0.004,--,--,--,--,--,--,-9.6,2.4,-2.9,2.4,0,--,--,--,9.1217917,-0.7286111,00 36 29.23,-00 43 43.0,10214,51,2,A0119
0.364831,9.12187,-0.728547,0.002,0.002,90,1237660355751379015,2,--,3,J003629.24-004342.7,*,35253360136208,2002.8535,3,17.731,0.015,15.905,0.003,15.09,0.003,14.695,0.003,14.393,0.004,--,--,--,--,--,--,-9.4,2.5,-2.8,2.5,0,--,--,--,9.1217917,-0.7286111,00 36 29.23,-00 43 43.0,10214,51,2,A0119
0.373178,9.121877,-0.728553,0.002,0.002,90,1237660025044598832,2,--,3,J003629.25-004342.7,*,35255507619856,2002.7772,3,17.749,0.018,15.893,0.003,15.063,0.003,14.682,0.003,14.349,0.004,--,--,--,--,--,--,-8.9,2.5,-3.5,2.5,0,--,--,--,9.1217917,-0.7286111,00 36 29.23,-00 43 43.0,10214,51,2,A0119
0.380413,9.121881,-0.728555,0.002,0.002,0,1237653011366739991,2,--,3,J003629.25-004342.7,*,35253360136208,2000.754,3,17.805,0.019,15.902,0.003,15.094,0.003,14.705,0.003,14.397,0.004,--,--,--,--,--,--,-8.8,2.6,-3.6,2.6,0,--,--,--,9.1217917,-0.7286111,00 36 29.23,-00 43 43.0,10214,51,2,A0119
